## Fonctions pour projet

need to normalize data ? (cf Dissecting financial markets: Sectors and states) (on devrait utiliser les returns donc pas trop besoin de normaliser)
- si on cluster les assets, logiquement la correlation/covariance entre chaque cluster devrait être négative/nulle (parce que la distance serait sans doute la covariance entre les assets), du coup faire une stratégie qui investit dans un représentant (asset ou weigted average) de chaque cluster devrait réduire le risque/volatilité du portfolio (problème étant que on aurait pas un zero-cost portfolio), ou alors prédire le signe de return de chaque cluster à la prochaine période et donc investir en fonction)
- si on cluster par jour/période de temps, l'idée serait de prédire le futur état et pour chaque état définir une méthode d'investissement
- https://d-nb.info/1108447864/34 papier qui parle de cluster les assets: truc important à noter est que les weights pour chaque cluster sont définis par hierarchical clustering p47-48, et les clusters/weights sont recalculés tous les 130 jours (permet d'éviter de refaire les clusters à chaque fois, et en pls ne suppose plus que les clusters restent les memes)

In [1]:

#Pour faire des clusters
#ce qui doit etre cluster sont les colonnes du dataframe (donc si on veut cluster les dates il faut transposer la matrice)


from numpy import linalg as LA
import numpy as np
import pandas as pd
import math
import networkx as nx
import community
import dask
dask.config.set(scheduler='processes')
import glob

    

In [2]:
#Idea is to remove low eigenvalues that are considered as noise
def compute_C_minus_C0(lambdas,v,lambda_plus,removeMarketMode=True): 
    N=len(lambdas)
    C_clean=np.zeros((N, N))
    
    order = np.argsort(lambdas)
    lambdas,v = lambdas[order],v[:,order]
    
    v_m=np.matrix(v)

    # note that the eivenvalues are sorted
    for i in range(1*removeMarketMode,N):                            
        if lambdas[i]>lambda_plus: 
            C_clean=C_clean+lambdas[i] * np.dot(v_m[:,i],v_m[:,i].T)  
    return C_clean    

In [3]:
def LouvainCorrelationClustering(R):   # R is a matrix of return
    N=R.shape[1]
    T=R.shape[0]

    q=N*1./T
    lambda_plus=(1.+np.sqrt(q))**2

    C=R.corr()
    lambdas, v = LA.eigh(C)


            
    C_s=compute_C_minus_C0(lambdas,v,lambda_plus)
    
    mygraph= nx.from_numpy_matrix(np.abs(C_s))
    partition = community.community_louvain.best_partition(mygraph,random_state=29)

    DF=pd.DataFrame.from_dict(partition,orient="index",columns=['Cluster'])
    DF.index=R.columns
    return(DF)

peut etre qu'on peut faire une analyse des eigenvalues et eigenvectors (cf cours 9-10)
mais la question est qu'on peut pas faire les eigenvalues sur tout le dataset (parce que dans ce cas on analyserait le futur avant meme de faire des clusters par exemple...)

De plus, cf cours 11 on pourrait faire varier les méthodes de cluster, que ce soit en changeant la facon de faire le cluster ou de changer le critère qui doit être cluster (return, sign of return, excess return ...)

et faudrait aussi du coup faire des investment strategies et les comparer entre elles, et avec un benchmark

In [4]:
from os.path import exists
import os
from fastparquet import write, ParquetFile

#Check if we already have the clustering or not and compute louvain cluster
@dask.delayed
def get_Louvain_cluster(R,filename,t_0,t_1, day_state=False) : 
    
    # t_0 and t_1 = window
    #day_state = TRUE -> market state clustering
    
    #Check if cluster already exists
    if(exists(filename) and os.path.getsize(filename)>0):
        
        #Import clustering
        DF=pd.read_parquet(filename)
        
    else:
        
        #Do clustering method and saves it
        rolling_data=R.iloc[(R.index>=t_0) & (R.index<=t_1)]
        
        if day_state:
            rolling_data=rolling_data.T
            
        DF= LouvainCorrelationClustering(rolling_data)
        DF.to_parquet(filename)
        
    return(DF)



In [18]:
@dask.delayed
def RolledCluster(R, cluster_period, filepath, day_state=False, keep_all_info=False, lag_max=20):
    
    #R = dataframe
    #cluster_period = length of rolling window
    #day_state = TRUE -> market state clustering
    #keep_all_info = TRUE -> rolling window expanding instead of just rolling (pas sur detre utile)
    #lag_max = number of clustering
    
    
    #Create list of clusters
    liste=[]
    
    #
    if keep_all_info: 
        if day_state :
            for lag in range(1,lag_max):
                
                #initilisation of window
                t_0=R.index[0]
                t_1=R.index[0+lag+cluster_period]   
                
                #compute convenient filename
                filename=filepath+"/{}_{}_cluster_day_keep_all_info.parquet".format(t_0,t_1)
                
                #compute cluster
                DF= get_Louvain_cluster(R,filename,t_0,t_1, day_state)
                
                #add cluster to list
                liste.append(DF)
        else :
            for lag in range(1,lag_max):  
                
                t_0=R.index[0]
                t_1=R.index[0+lag+cluster_period]    
                
                filename=filepath+"/{}_{}_cluster_keep_all_info.parquet".format(t_0,t_1)
        
                DF= get_Louvain_cluster(R,filename,t_0,t_1,day_state)
                liste.append(DF)
    else : 
        if day_state :
            for lag in range(1,lag_max):  
                
                t_0=R.index[0+lag]
                t_1=R.index[0+lag+cluster_period] 
                
                filename=filepath+"/{}_{}_cluster_day.parquet".format(t_0,t_1)
                
                DF= get_Louvain_cluster(R,filename,t_0,t_1,day_state)
                liste.append(DF)
        else :
            for lag in range(1,lag_max):  
                
                t_0=R.index[0+lag]
                t_1=R.index[0+lag+cluster_period]   
                
                filename=filepath+"/{}_{}_cluster.parquet".format(t_0,t_1)
        
                DF= get_Louvain_cluster(R,filename,t_0,t_1,day_state)
                liste.append(DF)
    return(liste)

#Renvoie une liste de clusters
#Choose the assets (thus N ), choose the calibration length 
#(T = N/3 is a good choice for market states, T = 3N is a good choice for asset classification)

In [6]:
#Compute the rolling ARI of the clusters
from sklearn import metrics

def RolledARI(liste):
    
    #Create list of ARI
    ARI=[]
    
    
    for element in range(0, len(liste)-1):
        
        #Compare consecutive clustering and add measure
        new_df=pd.merge(liste[element], liste[element+1], left_index=True, right_index=True)
        ARI.append(metrics.adjusted_rand_score(new_df["Cluster_x"], new_df["Cluster_y"]))
        
        
    return(ARI)

In [7]:
#fonction qu'on avait fait mais pas sur que ca serve dans notre cas
def compute_R(events,tau_max=1000,dtau=1):
    taus=range(1,tau_max,dtau)
    
    R=[-(events["mid"].diff(-tau)*events["s"]).mean() for tau in taus]
    
    
    return np.array(R)

def compute_R_h(events):
    R=pd.pivot_table(events.groupby(events.index.hour).apply(compute_R).apply(pd.Series), columns=events.groupby(events.index.hour).apply(compute_R).apply(pd.Series).index)
    return (R)

In [17]:

#pour linstant renvoie simplement une matrice de similarite
#l'entree (i,j) correspond au pourcentage d'éléments du cluster 2j qui appartiennent aussi au cluster 1i
def keep_cluster_number(cluster1,cluster2):
    
    #rename clusters based on similarity
    
    #compute number of clusters within each clustering period
    n=cluster1["Cluster"].unique().max()
    m=cluster2["Cluster"].unique().max()
    
    #dummy constant to avoid changing twice clusters later
    cst=10000
    
    #Create matrix of similarity
    matrix = np.zeros((n+1,m+1))
    
    for i in range(n+1):
        for j in range(m+1):
            merged=pd.merge(cluster1,cluster2, left_index=True, right_index=True)
            
            #get number of days/ticker belonging to cluster x_i and y_j
            num= len(merged[(merged["Cluster_y"]==j) & (merged["Cluster_x"]==i)])
            
            #divide by number of elements in cluster y_j
            denom=len(merged[(merged["Cluster_y"]==j)])
            
            matrix[i][j]= num/denom
            #entry (i,j) is the % of elements in y_j coming from x_i
    
    #get a copy of cluster2 that we change
    new_cluster2=cluster2.copy(deep=True) 
    
    #inititialize lists of cluster values
    used_values=[]
    unrenamed_cluster=[]
    
    #array with number of element in each cluster
    tmp=[]
    for i in range(0,m+1):
        tmp.append(len(cluster2[cluster2["Cluster"]==i]))

    #transform into df
    dff=pd.DataFrame(data=tmp)

    #sort values by ascending number of element
    #relabel cluster based on most important (size) cluster
    
    dff=dff.sort_values(by=0,ascending=False)

    for i in dff.index:
        
        #get the position of the highest similarity
        value= np.argmax(matrix[:,i])
    
        #verify that this value is not already attributed to another cluster
        if not np.isin(value,used_values):
        
            value_cluster=value
        
            #update the used values
            used_values.append(value)
        
            #change the new clusters
            #add dummy constant to avoid changing twice clusters
            new_cluster2[new_cluster2["Cluster"]==i]= (value_cluster+cst)
        
        else:
            #update cluster that need to be allocated a number
            unrenamed_cluster.append(i)

    #all possible values for cluster number
    possible_values= range(0,m+1)

    for element in unrenamed_cluster:
    
        #get smallest unused value
        value_cluster= np.min(np.where(~np.isin(possible_values,used_values)))
    
        #update used values
        used_values.append(value)
    
        #change cluster number
        new_cluster2[new_cluster2["Cluster"]==element]= (value_cluster+cst)
    
    #substract cst to get actual clusters numbers
    new_cluster2=new_cluster2-cst
    
    
    return(new_cluster2)
    

In [9]:
#One time prediction of cluster belonging
#lines = period t
#col = period t+1

def one_step_pred(cluster):
    
    #get number of clusters
    m=cluster["Cluster"].unique().max()
    
    #create square matrix
    matrix = np.zeros((m+1,m+1))
    
    for p in range(len(cluster)-1):
        i=cluster["Cluster"].iloc[p]
        tmp=p+1
        j=cluster["Cluster"].iloc[tmp]
        
        #adds 1 when you go from state i to state j
        matrix[i][j]=matrix[i][j]+1
        
    #divide by total number of changes    
    matrix=matrix/(len(cluster)-1)
    
    #matrix is now a probability matrix
    #matrix(i,j) is the probability to go from state i to state j
    
    return(matrix)

# Pour run les fonctions et les tester

In [29]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#logret= pd.read_parquet("data/clean/1m/data_clean_1m.parquet")


In [30]:
#lii= RolledCluster(logret,150,"/Users/mrobaux/Desktop/Financial big data/projet",True,False)

#number_of_clusters=[]
#for element in lii:
#    number_of_clusters.append(element["Cluster"].unique().max()+1)

# Test

In [31]:
#Idee: faire que les numeros de cluster restent consistants au cours du temps

#for i in range(len(lii)-1):
#    lii[i+1]=keep_cluster_number(lii[i],lii[i+1])



In [32]:
#matrix

In [33]:
#Create sankey plot
#!pip install plotly
#Pour voir comment ca marche
#Affiche un graphe de transition donc c'est plutot cool

#import plotly.graph_objects as go
#label = ["Cluster 0","Cluster 1","Cluster 2","Cluster 3","Cluster 0","Cluster 1","Cluster 2","Cluster 3","Cluster 0","Cluster 1","Cluster 2","Cluster 3","Cluster 0","Cluster 1","Cluster 2","Cluster 3"]
#source = [0,0,0,0,1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4,5,5,5,5,6,6,6,6,7,7,7,7,8,8,8,8]
#target =[4,5,6,7,4,5,6,7,4,5,6,7,4,5,6,7,8,9,10,11,8,9,10,11,8,9,10,11,8,9,10,11]
#value = [matrix[0][0],matrix[0][1],matrix[0][2],matrix[0][3],matrix[1][0],matrix[1][1],matrix[1][2],matrix[1][3],matrix[2][0],matrix[2][1],matrix[2][2],matrix[2][3],matrix[3][0],matrix[3][1],matrix[3][2],matrix[3][3],matrix1[0][0],matrix1[0][1],matrix1[0][2],matrix1[0][3],matrix1[1][0],matrix1[1][1],matrix1[1][2],matrix1[1][3],matrix1[2][0],matrix1[2][1],matrix1[2][2],matrix1[2][3],matrix1[3][0],matrix1[3][1],matrix1[3][2],matrix1[3][3]]
#link=dict(source=source,target=target,value=value)
#node= dict(label=label,pad=35, thickness=10)


#data=go.Sankey(link=link, node=node)


In [34]:
#go.Figure(data)

In [35]:
#plot d'un cluster vs un autre pour voir si on obtient qqc d'intéressant

#test=lii[1]
#index0= test[test["Cluster"]==1].index
#index1= test[test["Cluster"]==2].index


#mean0=logret.loc[index0].mean()
#mean1=logret.loc[index1].mean()

#plt.scatter(mean0,mean1)


# Brouillon

In [36]:
#!pip install pydot
#!pip install graphviz

In [37]:
#Supposer afficher un sankey hyper style mais ca veut pas mdr
#states = ["0","1","2","3"]
#def _get_markov_edges(Q):
#    edges = {}
#    for col in Q.columns:
#        for idx in Q.index:
#            edges[(idx,col)] = Q.loc[idx,col]
#    return edges

#edges_wts = _get_markov_edges(prob_state_transitions)

# create graph object
#G = nx.MultiDiGraph()

# nodes correspond to states
#G.add_nodes_from(states)

# edges represent transition probabilities
#for k, v in edges_wts.items():
#    tmp_origin, tmp_destination = k[0], k[1]
#    G.add_edge(tmp_origin, tmp_destination, weight=v, label=v)
#, prog='dot'
#nx.drawing.nx_pydot.write_dot(G,path=os.environ["PATH"])
#pos = nx.drawing.nx_pydot.pydot_layout(G,prog="dot")
#nx.draw_networkx(G, pos)

# create edge labels for jupyter plot but is not necessary
#edge_labels = {(n1,n2):d['label'] for n1,n2,d in G.edges(data=True)}
#nx.draw_networkx_edge_labels(G , pos, edge_labels=edge_labels)
#nx.drawing.nx_pydot.write_dot(G, 'pet_dog_markov.dot')

In [38]:
#Supposer afficher un plot sympa mais ca veut pas mdr


#weigted_edges=[]
#for i in range(4):
#    for j in range(4):
#        weigted_edges.append((i,j,prob_state_transitions[i][j]))


# create graph object
#G = nx.MultiDiGraph()

# nodes correspond to states
#G.add_nodes_from(states)
#G.add_weighted_edges_from(weigted_edges)

#nx.draw(G)